In [ ]:
import sys
sys.path.append('../../..')

from src.op_analytics.datasources.defillama.tokenprice.execute import (
    execute_pull_current,
    execute_pull_historical,
    execute_pull_first_prices,
)
from src.op_analytics.datasources.defillama.tokenprice.price_data import DefiLlamaTokenPrices
from op_analytics.datapipeline.chains.tokens import get_token_ids_from_metadata
from op_analytics.coreutils.request import new_session
from datetime import datetime, timedelta
import polars as pl

# Optional: Set up logging
import logging
logging.basicConfig(level=logging.INFO)


In [ ]:
# Get token IDs from chain metadata
token_ids = get_token_ids_from_metadata()
print(f"Found {len(token_ids)} token IDs from chain metadata")
print("First 10 token IDs:", token_ids[:10])


In [ ]:
session = new_session()

# Get current prices for multiple tokens
tokens = ["bitcoin", "ethereum", "ethereum:0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48"]
price_data = DefiLlamaTokenPrices.fetch_prices_current(tokens, session=session)

print(f"Fetched {len(price_data)} current prices")
print(price_data.df.head())

# Get historical prices for the last 7 days
price_data = DefiLlamaTokenPrices.fetch_prices_by_days(
    token_ids=["bitcoin", "ethereum"],
    days=7,
    session=session
)

print(f"\\nFetched {len(price_data)} historical prices")
print(price_data.df.head())

# Basic analysis
print("\\nPrice summary:")
print(price_data.df.group_by("token_id").agg([
    pl.col("price_usd").min().alias("min_price"),
    pl.col("price_usd").max().alias("max_price"),
    pl.col("price_usd").mean().alias("avg_price"),
    pl.col("dt").count().alias("data_points")
]))
